```python
LabelList.process??

Signature:
LabelList.process(
    ['self', 'xp: fastai.data_block.PreProcessor = None', 'yp: fastai.data_block.PreProcessor = None', 'name: str = None'],
)
Source:   
    def process(self, xp:PreProcessor=None, yp:PreProcessor=None, name:str=None):
        "Launch the processing on `self.x` and `self.y` with `xp` and `yp`."
        self.y.process(yp)
        if getattr(self.y, 'filter_missing_y', False):
            filt = array([o is None for o in self.y.items])
            if filt.sum()>0:
                #Warnings are given later since progress_bar might make them disappear.
                self.warn = f"You are labelling your items with {self.y.__class__.__name__}.\n"
                self.warn += f"Your {name} set contained the following unknown labels, the corresponding items have been discarded.\n"
                for p in self.y.processor:
                    if len(getattr(p, 'warns', [])) > 0:
                        warnings = list(set(p.warns))
                        self.warn += ', '.join(warnings[:5])
                        if len(warnings) > 5: self.warn += "..."
                    p.warns = []
                self.x,self.y = self.x[~filt],self.y[~filt]
        self.x.process(xp)
        return self
File:      ~/Documents/fastai-contrib/fastai/data_block.py
Type:      function
```

Behind the scenes, `LabelList.process` does 3 three things: 1. ask labels `y` to be processed by `yp` with `y.process(yp)`; 2. if `y.filter_missing_y` is `True`, then removes the missing data samples from `x` and `y`; 3. ask inputs `x` to be processed by `xp` with `x.process(xp)`

In [17]:
from fastai.vision import *

In [18]:
path_data = untar_data(URLs.MNIST_TINY)
sd = ImageList.from_folder(path_data).split_by_folder('train', 'valid')

In [19]:
sd.train = sd.train.label_from_folder(from_item_lists=True)
sd.valid = sd.valid.label_from_folder(from_item_lists=True)
sd.__class__ = LabelLists

In [20]:
xp,yp = sd.get_processors()
xp,yp

([], [<fastai.data_block.CategoryProcessor at 0x1a23757a90>])

In [21]:
sd.train.process(xp, yp)

LabelList (709 items)
x: ImageList
Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28),Image (3, 28, 28)
y: CategoryList
7,7,7,7,7
Path: /Users/Natsume/.fastai/data/mnist_tiny